## DigiSafe
---

In [1]:
from Crypto.Cipher import ChaCha20_Poly1305
import argon2
import avro.datafile, avro.io
import base64
import hashlib
import io
import lz4.frame
import os
import reedsolo

In [2]:
def open_sesame(file_path="../Rust/test.digisafe", key="test_key"):
    key = hashlib.sha3_256(hashlib.sha3_256(key.encode()).digest()).digest()
    key = argon2.low_level.hash_secret_raw(key, b"digisafe", time_cost=2, memory_cost=2**20, parallelism=8, hash_len=32, type=argon2.low_level.Type.ID)
    key = hashlib.sha3_256(hashlib.sha3_256(key).digest()).digest()
    db_raw_enc = open(file_path, "rb").read()
    db_avro_enc = reedsolo.RSCodec(8).decode(db_raw_enc)[0]
    db_enc = [*avro.datafile.DataFileReader(io.BytesIO(db_avro_enc), avro.io.DatumReader())][0]
    db_lz4 = ChaCha20_Poly1305.new(key=key, nonce=base64.b64decode(db_enc["meta"]["nonce"])).decrypt(base64.b64decode(db_enc['db']))
    db_avro = lz4.frame.decompress(db_lz4)
    db = [*avro.datafile.DataFileReader(io.BytesIO(db_avro), avro.io.DatumReader())][0]
    return db

In [3]:
db = open_sesame()
db

{'db': {'_name': 'main', 'k3': 'v3', 'k2': 'v2', 'k1': 'v1'},
 'meta': {'uid': 'Kb+kH90y8DyIX+9fnEnt208re4fCNbKOcfZaB1jGOFM=',
  'version': '1.0.0',
  'timestamp': '1699588749',
  'app': 'digisafe',
  'revision': '0'}}